In [1]:
#importing  packages
import re
import string
import collections
import math as m

In [2]:
#!pip install cltk==v0.1.99       #installing cltk
from cltk.tokenize.sentence import TokenizeSentence
tokenizer = TokenizeSentence('telugu')


In [3]:
path='/home/karthik/Desktop/keeru/datasets/tewiki_all.txt'    
with open(path) as myfile:
    lines = [line.split('. ') for line in myfile.readlines()]
testlines = lines[450000:500000]
lines = lines[:450000]

In [4]:
#function removing the html tags,punctuations,numbers,english words
def preprocessing_data(str):           
    u = re.sub('<[^<]+?>','',str)
    o = re.sub('\u200c',' ',u)
    table = str.maketrans({key: None for key in string.punctuation})
    translated = o.translate(table)
    w = re.sub('[a-zA-Z]*','',translated)
    d = re.sub(' +',' ',w)
    #x = re.sub('[0-9]*','',d)
    y = re.sub("\n",'',d)
    return y

#getting a list of sentences that are clean
def get_preprocessed_sentences(sentences):    
    tokenized_sentences=[]
    for i in range(0,len(sentences)):
        temp = []
        for j in range(0,len(sentences[i])):
            final = preprocessing_data(sentences[i][j])
            if len(final) != 0:
                temp.append(final)
        tokenized_sentences.append(temp)
    return tokenized_sentences

train_preprocessed_sentences = get_preprocessed_sentences(lines)
test_preprocessed_sentences = get_preprocessed_sentences(testlines)

In [5]:
#function to tokenize the sentences
def getting_sentences(line):               
    sentences = []
    for i in range(0,len(line)):
        for j in range(0,len(line[i])):
            sentence = line[i][j]
            s = tokenizer.tokenize(sentence)
            sentences.append(s)
    return sentences
train_sentences = getting_sentences(train_preprocessed_sentences)
test_sentences = getting_sentences(test_preprocessed_sentences)

In [6]:
# adding start and end delimiter, then splitting
def delimiting_spliting(tokenized_sentences):   
    tokenized_sentences = [x for x in tokenized_sentences if x != ['\n'] and x != [' \n']]
    sentences_with_delimiter = []
    for i in range(0,len(tokenized_sentences)):
        s = tokenized_sentences[i]
        if len(s) == 0:
            continue
        temp = ['<s> ']+s+[' </s>']                #adding start and end delimiter
        sentences_with_delimiter.append(temp)
    print(sentences_with_delimiter[:10])
    #split_data =[]
    #for i in range(0,len(sentences_with_delimiter)):
        #d = sentences_with_delimiter[i].split()
        #split_data.append(d)                #spliting the words, and getting the list that contain list of words of every sentence
    return sentences_with_delimiter
split_train_data = delimiting_spliting(train_sentences)
split_test_data = delimiting_spliting(test_sentences)

[['<s> ', 'మొదటి', 'పేజీ', ' </s>'], ['<s> ', 'గుంటూరు', 'జిల్లా', ' </s>'], ['<s> ', 'గుంటూరు', 'జిల్లా', '11391', 'చకిమీ', ' </s>'], ['<s> ', 'ల', 'విస్తీర్ణములో', 'వ్యాపించి', '4889230', '2011', 'గణన', 'జనాభా', 'కలిగిఉన్నది', ' </s>'], ['<s> ', 'ఆగ్నేయాన', 'బంగాళాఖాతము', 'దక్షిణాన', 'ప్రకాశం', 'జిల్లా', 'పశ్చిమాన', 'మహబూబ్', 'నగర్', 'జిల్లా', 'మరియు', 'వాయువ్యాన', 'నల్గొండ', 'జిల్లా', 'సరిహద్దులుగా', 'ఉన్నాయి', ' </s>'], ['<s> ', 'దీని', 'ముఖ్యపట్టణం', 'గుంటూరు', ' </s>'], ['<s> ', 'ఈ', 'జిల్లాకు', 'అతి', 'పురాతన', 'చరిత్ర', 'ఉంది', ' </s>'], ['<s> ', 'మౌర్యులు', 'శాతవాహనులు', 'పల్లవులు', 'చాళుక్యులు', 'కాకతీయులు', 'రెడ్డి', 'రాజులు', 'విజయనగర', 'రాజులు', 'పరిపాలించారు', ' </s>'], ['<s> ', 'పల్నాటి', 'యుద్ధం', 'ఇక్కడే', 'జరిగింది', ' </s>'], ['<s> ', 'మొగలు', 'సామ్రాజ్యం', 'నిజాం', 'పాలన', 'ఈస్ట్', 'ఇండియా', 'కంపెనీ', 'ఆ', 'తరువాత', 'మద్రాసు', 'ప్రసిడెన్సీలో', 'భాగమైనది', ' </s>']]
[['<s> ', 'గ్రామంలో', 'కుళాయిల', 'ద్వారా', 'రక్షిత', 'మంచినీటి', 'సరఫరా', 'జరుగుతోంది', ' </s>'], ['<s

In [7]:
#getting a single list of words
def getting_words_list(split_data):      
    list = [item for sublist in split_data for item in sublist]
    return list

In [8]:
words_train = getting_words_list(split_train_data)   #train data words
words_test = getting_words_list(split_test_data)     #test data words

In [9]:
unique_words = len(set(words_train)) # getting number of unique words in train data

In [10]:
#geting the dictionary(model) from the train data, where keys are the history words and values are 
#another dictionary which contains keys as the next word and its values corresponding to the number 
#of times that words has occurred

dic = {}
for i in range(0,len(words_train)-1):
    t = words_train[i]
    s = dic.get(t)
    next = i+1
    if s == None:
        dic[t] = [words_train[next]]
    else:
        s.append(words_train[next])
        temp = {t:s}
        dic.update(temp)
#test_dic is a dictionary where keys are the history words of the test data and values is the list of next words     

test_dic = {}
for i in range(0,len(words_test)-1):
    t = words_test[i]
    s = test_dic.get(t)
    next = i+1
    if s == None:
        test_dic[t] = [words_test[next]]
    else:
        s.append(words_test[next])
        temp = {t:s}
        test_dic.update(temp)


In [11]:
# getting the count of each element in the list
def frequency_count(list):
    c = collections.Counter(list)
    count_dic={}
    for i in range(0,len(list)):
        count_dic[list[i]] = c[list[i]]
    return count_dic


In [12]:
# updating the dic with the word frequency
for word in dic:
    s = dic.get(word)
    value = frequency_count(s)
    temporary = {word:value}
    dic.update(temporary)
    

In [13]:
# getting the probability of each predicted word
def get_probability(s1,s2):
    firstWord = s1
    nextWord = s2
    inner_dic = dic.get(firstWord)
    
    if inner_dic == None:
        return 0
    else:
        numerator = inner_dic.get(nextWord)
        if numerator == None:
            return 0
        else:
            denominator = sum(inner_dic.values())
            return numerator/denominator

In [14]:
# getting the add-one probability of each predicted word
def get_addOne_probability(s1,s2):
    firstWord = s1
    nextWord = s2
    inner_dic = dic.get(firstWord)
    if inner_dic == None:
        return 0 
    else:
        numerator = inner_dic.get(nextWord)
        if numerator == None:
            numerator = 1
        else:
            numerator = numerator + 1
        denominator = sum(inner_dic.values()) + unique_words
    return numerator/denominator

In [15]:
# getting the add-alpha probability of each predicted word
def get_addAlpha_probability(s1,s2):
    firstWord = s1
    nextWord = s2
    alpha = 0.1
    inner_dic = dic.get(firstWord)
    if inner_dic == None:
        return 0 
    else:
        numerator = inner_dic.get(nextWord)
        if numerator == None:
            numerator = alpha
        else:
            numerator = numerator + alpha
        denominator = sum(inner_dic.values()) + (unique_words*alpha)
    return numerator/denominator

In [16]:
#preparing probability dictionaries with corresponding probabilities
prob_dic = {}
addOneProb_dic = {}
addAlphaProb_dic = {}
for i in range(0,len(words_test)-2):
    bigram = words_test[i]+words_test[i+1]
    prob = get_probability(words_test[i],words_test[i+1])
    addOneProb = get_addOne_probability(words_test[i],words_test[i+1])
    prob_dic[bigram]=prob
    addOneProb_dic[bigram] = addOneProb
    addAlphaProb = get_addAlpha_probability(words_test[i],words_test[i+1])
    addAlphaProb_dic[bigram] = addAlphaProb



In [17]:
# computing perplexity Score
def perplexityScore(prob_dic):
    logProb = 0
    num = 0
    for word in prob_dic:
        p = prob_dic.get(word)
        if p == 0:
            logProb = logProb
        else:
            w = m.log(p**-1,10)
            logProb = logProb + w 
            num = num + 1
    return (2**(logProb/num))


In [18]:
# perplexity Scores for different cases
ans = perplexityScore(prob_dic)
print(ans)
ans1 = perplexityScore(addOneProb_dic)
print(ans1)
ans2 = perplexityScore(addAlphaProb_dic)
print(ans2)


4.275553930100999
33.564570220418624
28.40261011567869


In [19]:
#input interface
s1 = input("Enter the 1st word " )
input_word = s1
print(input_word)
def getting_2nd(input_word):
    s = dic.get(input_word)
    if s == None:
        print("not found")
        return
    else:
        maximum = max(s, key=s.get)
        return maximum
print(getting_2nd(input_word))
    

Enter the 1st word మహబూబ్
మహబూబ్
నగర్


In [20]:
#predicting a sequence of words
s1 = input("Enter the 1st word " )
input_word = s1
i = 1
next = getting_2nd(input_word)
for i in range(1,15):
        print(next)
        next2 = getting_2nd(next)
        next = next2
       

Enter the 1st word గుంటూరు
జిల్లా
రహదారి
గ్రామం
నుండి
10
కిమీ
 </s>
<s> 
గ్రామంలో
మగవారి
సంఖ్య
0
 </s>
<s> 


గుంటూరు
సరిహద్దులుగా
మహబూబ్